1.  One_hot_encoding for ordinal and categorial columns (rank information on education and poverty is lo
2.  Check for NAs 
    -> delete employment columns, health insurance
    -> delete rows with no information on recommandation of doctor (poor answering behavior) 
    -> take rows of opinion columns and impute NAs with k nearest neighbour 
3.  Delete the same rows in the labels dataset
    

In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
# preprocess raw data
# split raw data into train and eval
# log_reg fit 
# preprocess test data 
# log_reg fit
# submit

In [3]:
data_path = Path.cwd().parent.parent / "data" / "raw"
raw_df = pd.read_csv(data_path / "training_set_features.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path / "training_set_labels.csv", index_col="respondent_id")
test_df = pd.read_csv(data_path / "test_set_features.csv", index_col="respondent_id")

all_df = raw_df.join(labels_df)
#all_df.columns

In [4]:
pd.set_option("display.max_rows", 20, "display.max_columns", 20)

In [5]:
# one-hot-enconding without employment_occuptation (is employment_occuptation either dropped or Weight of Evidence Encoded)
    # drop column employment_occupation 
    # identify columns with categorial data 
    # just use columns with nominal data
    # getdummies() -> drop first column to dismiss multicollinarity, keep NaN column
raw_df = raw_df.drop(columns = ['employment_occupation'])

non_numerical_obj = raw_df.columns[raw_df.dtypes == "object"]
non_numerical_obj

nominal_list = ['race', 'sex', 
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry']

raw_df_encoded = pd.get_dummies(raw_df, columns=nominal_list, drop_first = True, dummy_na=True)
test_df_encoded = pd.get_dummies(test_df, columns=nominal_list, drop_first = True, dummy_na=True) 

In [6]:
# Weight of Evidence Encoding of Occuptation on raw_data
# 'h1n1_vaccine', 'seasonal_vaccine' 
def WOE(var, tar):
    all_df[var] = all_df[var].fillna('NoData')
    k = all_df[[var,tar]].groupby(var)[tar].agg(['count','sum']).reset_index()
    k.columns = [var,'Count','Good']
    k['Bad'] = k['Count'] - k['Good']
    k['Good %'] = (k['Good'] / k['Good'].sum()*100).round(2)
    k['Bad %'] = (k['Bad'] / k['Bad'].sum()*100).round(2)
    k[var+tar+'_WOE'] = np.log(k['Good %'] / k['Bad %']).round(2)
    k = k.sort_values(by=var+tar+'_WOE')
    return(k)
h1n1_WOE = WOE('employment_occupation' , 'h1n1_vaccine')
seasonal_WOE = WOE('employment_occupation' , 'seasonal_vaccine')

WOE_df_season = pd.merge(all_df[['seasonal_vaccine','employment_occupation']],seasonal_WOE[['employment_occupation','employment_occupationseasonal_vaccine_WOE']],
     left_on='employment_occupation',
     right_on='employment_occupation',how='left')
WOE_df_h1n1 = pd.merge(all_df[['h1n1_vaccine','employment_occupation']],h1n1_WOE[['employment_occupation','employment_occupationh1n1_vaccine_WOE']],
     left_on='employment_occupation',
     right_on='employment_occupation',how='left')


WOE_df_both = WOE_df_h1n1.join(WOE_df_season, lsuffix='_h1n1', rsuffix='_seasonal')
WOE_mean = WOE_df_both[['employment_occupationh1n1_vaccine_WOE', 
                           'employment_occupationseasonal_vaccine_WOE']].mean(axis = 1)

raw_df_encoded['employment_occupation_WOE'] = WOE_mean

In [7]:
# Weight of Evidence Encoding of Occuptation on test_data
# 'h1n1_vaccine', 'seasonal_vaccine' 
test_df_encoded['employment_occupation_new'] =test_df_encoded['employment_occupation'].replace(np.nan, 'NoData')

test_WOE_df_season = pd.merge(test_df_encoded['employment_occupation_new'],seasonal_WOE[['employment_occupation','employment_occupationseasonal_vaccine_WOE']],
     left_on='employment_occupation_new',
     right_on='employment_occupation',how='left')
test_WOE_df_season.index =  test_df_encoded.index

test_WOE_df_h1n1 = pd.merge(test_df_encoded[['employment_occupation_new']],h1n1_WOE[['employment_occupation','employment_occupationh1n1_vaccine_WOE']],
     left_on='employment_occupation_new',
     right_on='employment_occupation',how='left')
test_WOE_df_h1n1.index =  test_df_encoded.index


test_WOE_df_both = test_WOE_df_h1n1 .join(test_WOE_df_season, lsuffix='_h1n1', rsuffix='_seasonal')

test_WOE_df_both.index =  test_df_encoded.index


test_df_encoded['employment_occupation_WOE'] = test_WOE_df_both[['employment_occupationh1n1_vaccine_WOE', 
                           'employment_occupationseasonal_vaccine_WOE']].mean(axis = 1)



test_df_encoded = test_df_encoded.drop(columns = ['employment_occupation','employment_occupation_new' ])




In [8]:
# encoding of ordinal data 
    # identifiying ordinal data columns 
    # change NaN in ordinal data columns to most frequent value 
    # change strings in ordinal data to numbers 
    # delete values in columns wich where imputed with most frequent values
    # impute NaN values with KNN
    
ordinal_list = ['age_group','education', 'income_poverty' ]



In [9]:
# impute misssing values in ordinal data columns with most frequent value 


from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
raw_df_encoded_mode = imp_mode.fit_transform(raw_df_encoded[ordinal_list])
raw_df_encoded_mode_pd = pd.DataFrame(raw_df_encoded_mode, index = raw_df_encoded.index)

raw_df_encoded['age_group'] = raw_df_encoded_mode_pd[0]
raw_df_encoded['education'] = raw_df_encoded_mode_pd[1]
raw_df_encoded['income_poverty'] = raw_df_encoded_mode_pd[2]

test_imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
test_df_encoded_mode = test_imp_mode.fit_transform(test_df_encoded[ordinal_list])
test_df_encoded_mode_pd = pd.DataFrame(test_df_encoded_mode, index = test_df_encoded.index )

test_df_encoded['age_group'] = test_df_encoded_mode_pd[0]
test_df_encoded['education'] = test_df_encoded_mode_pd[1]
test_df_encoded['income_poverty'] = test_df_encoded_mode_pd[2]

In [10]:
# change strings in ordinal data to numbers 

from sklearn.preprocessing import OrdinalEncoder
ordinal_enc = OrdinalEncoder ( 
    categories = [
        ['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'],
        ['< 12 Years', '12 Years', 'Some College', 'College Graduate' ],
        ['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'],
     ]
)

raw_df_encoded_ordinal = ordinal_enc.fit_transform(raw_df_encoded[ordinal_list])
raw_df_encoded_ordinal = pd.DataFrame(raw_df_encoded_ordinal, raw_df_encoded.index)
#ordinal_enc.categories_

raw_df_encoded['age_group'] = raw_df_encoded_ordinal[0]
raw_df_encoded['education'] = raw_df_encoded_ordinal[1]
raw_df_encoded['income_poverty'] = raw_df_encoded_ordinal[2]

test_df_encoded_ordinal = ordinal_enc.fit_transform(test_df_encoded[ordinal_list])
test_df_encoded_ordinal = pd.DataFrame(test_df_encoded_ordinal, test_df_encoded.index)


test_df_encoded['age_group'] = test_df_encoded_ordinal[0]
test_df_encoded['education'] = test_df_encoded_ordinal[1]
test_df_encoded['income_poverty'] = test_df_encoded_ordinal[2]



In [11]:
#delete values in columns wich where imputed with most frequent values

education_encoded = raw_df_encoded['education'].copy()
education_na = raw_df['education'].isna()
education_encoded[education_na] = np.nan

raw_df_encoded['education'] = education_encoded

income_poverty_encoded = raw_df_encoded['income_poverty'].copy()
income_poverty_na = raw_df['income_poverty'].isna()
income_poverty_encoded[income_poverty_na] = np.nan

raw_df_encoded['income_poverty'] = income_poverty_encoded

test_education_encoded = test_df_encoded['education'].copy()
test_education_na = test_df['education'].isna()
test_education_encoded[test_education_na] = np.nan

test_df_encoded['education'] = test_education_encoded

test_income_poverty_encoded = test_df_encoded['income_poverty'].copy()
test_income_poverty_na = test_df['income_poverty'].isna()
test_income_poverty_encoded[test_income_poverty_na] = np.nan

test_df_encoded['income_poverty'] = test_income_poverty_encoded

In [12]:
##caculate KNN for the whole dataset  // either this or next cell


#import numpy as np
#from sklearn.impute import KNNImputer
#imputer = KNNImputer(
#    missing_values=np.nan,
#    n_neighbors=5,
#    weights="distance",
#    metric="nan_euclidean",
#    copy=True,
#    add_indicator=False,
#)

#imputer.fit(raw_df_encoded)

#knn_array = imputer.transform(raw_df_encoded)

#raw_df_encoded_imputed = pd.DataFrame(knn_array, raw_df_encoded.index)
#raw_df_encoded_imputed.columns = raw_df_encoded.columns

#change KNN imputations in ordinal data columns to "categories"

#raw_df_encoded_imputed['education'].round()
#raw_df_encoded_imputed['income_poverty'].round()

In [13]:
## alternativly to KNN impute missing values with NaN

from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='mean')
raw_df_encoded_imputed = imp_mode.fit_transform(raw_df_encoded)

raw_df_encoded_imputed = pd.DataFrame(raw_df_encoded_imputed, columns = raw_df_encoded.columns, index = raw_df_encoded.index)

raw_df_median = imp_mode.fit(raw_df_encoded)
test_df_encoded_imputed = imp_mode.transform(test_df_encoded)

test_df_encoded_imputed = pd.DataFrame(test_df_encoded_imputed, columns = test_df_encoded.columns, index = test_df_encoded.index)


In [14]:
# renaming
raw_df_encoded_imputed_list = raw_df_encoded_imputed.columns.tolist()

renamed_list = []
for column_name in raw_df_encoded_imputed_list:
    column_name = re.sub(pattern=r"\$(\d\d)....", repl="\\1k", string=column_name)
    column_name = re.sub(pattern=r"\<\=|\<\s", repl="less_", string=column_name)
    column_name = re.sub(pattern=r"\>\=|\>\s", repl="gr_", string=column_name)
    column_name = re.sub(pattern=r"(\w)\,\s(\w)", repl="\\1_\\2", string=column_name)
    column_name = re.sub(pattern=r"\s\-\s", repl="_-_", string=column_name)
    column_name = re.sub(pattern=r"(\w)\s+(\w)", repl="\\1_\\2", string=column_name)
    column_name = re.sub(pattern=r"(\d\+)\s", repl="\\1_", string=column_name)
    column_name = re.sub(pattern=r"\_\_", repl="_", string=column_name)
    renamed_list.append(column_name)
    
raw_df_encoded_imputed.columns = renamed_list
test_df_encoded_imputed.columns = renamed_list

In [15]:
#data_path_processed = Path.cwd().parent.parent / "data" / "processed"
#raw_df_encoded.to_csv(data_path_processed / "training_set_features_1hot_na.csv",)
#test_df_encoded.to_csv(data_path_processed / "test_set_features_1hot_na.csv",)

In [16]:
# check for multi collinarity 

pd.set_option("display.max_rows", None, "display.max_columns", None)
corr_data = raw_df_encoded_imputed.corr().abs()
sorted_corr_data = corr_data.unstack().sort_values(ascending=False)
ones = corr_data.unstack().sort_values(ascending=False) != 1.0
without_ones = sorted_corr_data[ones]
NaNs = corr_data.unstack().sort_values(ascending=False).notna()

without_ones_and_Na = without_ones[NaNs]
big_corrs = corr_data.unstack().sort_values(ascending=False) > 0.5
without_ones_and_Na[big_corrs]

marital_status_nan                    employment_status_nan                   0.876134
employment_status_nan                 marital_status_nan                      0.876134
employment_status_Not_in_Labor_Force  employment_industry_nan                 0.789401
employment_industry_nan               employment_status_Not_in_Labor_Force    0.789401
rent_or_own_nan                       employment_status_nan                   0.770415
employment_status_nan                 rent_or_own_nan                         0.770415
rent_or_own_nan                       marital_status_nan                      0.742348
marital_status_nan                    rent_or_own_nan                         0.742348
health_worker                         employment_industry_fcxhlnwr            0.694224
employment_industry_fcxhlnwr          health_worker                           0.694224
doctor_recc_seasonal                  doctor_recc_h1n1                        0.591868
doctor_recc_h1n1                      docto

In [22]:
raw_pd_df_encoded_imputed_stand.isna().sum().sort_values(ascending = False)

employment_status_Not in Labor Force    26707
marital_status_Not Married              26707
census_msa_MSA, Principle City          26707
race_Other or Multiple                  26707
household_children                          0
opinion_seas_sick_from_vacc                 0
age_group                                   0
education                                   0
income_poverty                              0
household_adults                            0
race_Hispanic                               0
opinion_seas_vacc_effective                 0
race_White                                  0
race_nan                                    0
sex_Male                                    0
sex_nan                                     0
marital_status_nan                          0
opinion_seas_risk                           0
opinion_h1n1_sick_from_vacc                 0
rent_or_own_nan                             0
opinion_h1n1_risk                           0
h1n1_knowledge                    

In [18]:
#standardize dataset

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
raw_df_encoded_imputed_stand = scaler.fit_transform(raw_df_encoded_imputed)
raw_df_encoded_imputed_stand = pd.DataFrame(raw_df_encoded_imputed_stand, columns = raw_df_encoded_imputed.columns, index = raw_df_encoded_imputed.index)

test_df_encoded_imputed_stand = scaler.fit_transform(test_df_encoded_imputed)
test_df_encoded_imputed_stand = pd.DataFrame(test_df_encoded_imputed_stand, columns = test_df_encoded_imputed.columns, index = test_df_encoded_imputed.index)

In [26]:
data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"

raw_pd_df_encoded_imputed_stand = pd.DataFrame(raw_df_encoded_imputed_stand, columns = raw_df_encoded.columns, index = raw_df_encoded.index)
raw_df_encoded_imputed_stand.to_csv (data_path_preprocessed / 'training_set_features_Preprocessing_nominal_ordinal_WOE_Impute_Stand.csv')

In [27]:
data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"

#test_pd_df_encoded_imputed_stand = pd.DataFrame(test_df_encoded_imputed_stand, columns = test_df_encoded.columns, index = test_df_encoded.index)
test_df_encoded_imputed_stand.to_csv (data_path_preprocessed / 'test_set_features_Preprocessing_nominal_ordinal_WOE_Impute_Stand.csv')